### Hybrid Search

In [ ]:
!pip install --upgrade --quite pinecone-client pinecone-text pinecone-notebooks

In [ ]:
api_key = ""

In [ ]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

In [ ]:
import os
from pinecone import Pinecone, ServerlessSpec

index_name = "hybrid-search-langchain-pinecone"

# initialize pinecone client
pc = Pinecone(api_key=api_key)

# Create index
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,   # Dimension of dense vector
        metric='dotproduct',        ## Sparse values supported only for dotproduct
        spec=ServerlessSpec(cloud='aws', region="us-east-1")
    )

In [ ]:
index = pc.Index(index_name)
index

In [6]:
import os
from dotenv import load_dotenv

load_dotenv()

from langchain_huggingface import HuggingFaceEmbeddings

os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings

HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [ ]:
from pinecone_text.sparse import BM25Encoder

bm25_encoder = BM25Encoder().default()
bm25_encoder

In [ ]:
sentences = [
    "In 2023, I visited new york",
    "In 2022, I visited Paris",
    "In 2021, I visited Orleans"
]

# TFIDF values on these encoders
bm25_encoder.fit(sentences)

# Store the values to a json file
bm25_encoder.dump("bm25_values.json")

bm25_encoder = BM25Encoder().load("bm25_values.json")

In [ ]:
retriever = PineconHybridSearchRetriever(embeddings=embeddings, spase_encoder=bm25_encoder, index=index)

retriever

In [ ]:
retriever.add(sentences)

In [ ]:
retriever.invoke("What city did I visit last")

In [ ]:
retriever.invoke("What city did I visit recent")

In [ ]:
retriever.invoke("What city did I visit first")